In [2]:

# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-6311  GCE       4                                       RUNNING  us-central1-a


In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:

# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  9 10:30 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [7]:

spark

In [8]:

# Put your bucket name below and make sure you can access it without an error
bucket_name = '205915135'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

Inverted Index

In [9]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [10]:

# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [11]:
from inverted_index_gcp import *
TUPLE_SIZE = 6
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w,file_name):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    locs = [(file_name + lo[0], lo[1]) for lo in locs]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
#   raise NotImplementedError()
  tf=Counter(tokens)
  return [(token, (id, tf[token])) for token in tf if token not in all_stopwords]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  return sorted(unsorted_pl, key=lambda x: x[0])
def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  return postings.map(lambda x: (x[0], len(x[1])))
  
def partition_postings_and_write(postings ):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  #raise NotImplementedError()
  # partition the posting lists into buckets
  postings = postings.map(lambda x: (token2bucket_id(x[0]), x))
  # write out all posting lists in a bucket to disk
  postings = postings.groupByKey()
  postings = postings.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))
  # return the posting locations for each bucket
  return postings

In [14]:
import builtins

def calculate_term_total(postings):
    ''' Takes a posting list RDD and calculate the term total for each token.
    Parameters:
    -----------
    postings: RDD
        An RDD where each element is a (token, posting_list) pair.
    Returns:
    --------
    RDD
        An RDD where each element is a (token, term_total) pair.
    '''
    return postings.mapValues(lambda posting: builtins.sum([tf for doc_id, tf in posting]))

In [15]:
from pyspark.shell import spark

parquetFile = spark.read.parquet(*paths)

In [16]:
parquetFile.count()


6348910

## Title


In [16]:
doc_text_pairs_title = parquetFile.select("title", "id").rdd

In [17]:
#title
word_counts_title = doc_text_pairs_title.flatMap(lambda x: word_count(x[0], x[1]))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)

In [18]:
w2df_title = calculate_df(postings_title)
w2df_dict = w2df_title.collectAsMap()

w2terms = calculate_term_total(postings_title)
dict_term_total = w2terms.collectAsMap()


In [19]:
# partition posting lists and write out
_ = partition_postings_and_write(postings_title).collect()

In [21]:
# collect all posting lists locations into one super-set
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [23]:
# Create inverted index instance
inverted_title = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict
# Add the total terms
inverted_title.term_total=dict_term_total
# write the global stats out
inverted_title.write_index('.', 'title_index')

In [25]:
index_title=pd.read_pickle('/home/dataproc/title_index.pkl')

In [26]:
len(index_title.df)

1774265

## BODY

In [17]:
from tokenize import tokenize


def doc_len(id, text):
    ''' Calculate the length of the text
    Parameters:
    -----------
    id: int
        Document id
    text: str
        Text of one document
    Returns:
    --------
    A dict of (doc_id, doc_len) pairs 
    for example: {12:5, ...}
    '''
    tokenized = tokenize(text)
    result = []
    return [(id, len(tokenized))]

In [39]:
doc_text_pairs_body = parquetFile.select("text", "id").rdd

In [43]:

#body
word_counts_of_body = doc_text_pairs_body.flatMap(lambda x: word_count(x[0], x[1]))
postings_body = word_counts_of_body.groupByKey().mapValues(reduce_word_counts)
#doc lengh
dl=doc_text_pairs_body.flatMap(lambda x: doc_len(x[1], x[0])) 
postings_filtered_body = postings_body.filter(lambda x: len(x[1])>50)

In [ ]:
w2df_body = calculate_df(postings_filtered_body)
w2df_body = w2df_body.collectAsMap()

w2termsbody = calculate_term_total(postings_filtered_body)
dict_term_total_body = w2termsbody.collectAsMap()


_ = partition_postings_and_write(postings_filtered_body).collect()
# collect all posting lists locations into one super-set
super_posting_locs_body = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_body[k].extend(v)

In [47]:
inverted_body = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_body.posting_locs = super_posting_locs_body
# Add the token - df dictionary to the inverted index
inverted_body.df = w2df_body
#Add the total terms
inverted_body.term_total = dict_term_total_body
# Add the doc lenth
inverted_body.DL=dl.collectAsMap()
# write the global stats out
inverted_body.write_index('.','body_index')

23/01/10 02:04:23 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2.0 in stage 25.0 (TID 1834) (cluster-6311-w-1.c.stable-apogee-366911.internal executor 34): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_8953/4169666495.py", line 5, in <lambda>
  File "/tmp/ipykernel_8953/47135752.py", line 14, in doc_len
NameError: name 'tokenize' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(Python

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 9 in stage 25.0 failed 4 times, most recent failure: Lost task 9.3 in stage 25.0 (TID 1854) (cluster-6311-w-1.c.stable-apogee-366911.internal executor 34): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_8953/4169666495.py", line 5, in <lambda>
  File "/tmp/ipykernel_8953/4713

23/01/10 02:04:46 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 12.2 in stage 25.0 (TID 1852) (cluster-6311-w-3.c.stable-apogee-366911.internal executor 39): TaskKilled (Stage cancelled)
23/01/10 02:04:46 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 10.0 in stage 25.0 (TID 1846) (cluster-6311-w-1.c.stable-apogee-366911.internal executor 42): TaskKilled (Stage cancelled)
23/01/10 02:04:46 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 5.0 in stage 25.0 (TID 1839) (cluster-6311-w-3.c.stable-apogee-366911.internal executor 38): TaskKilled (Stage cancelled)
23/01/10 02:04:47 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 7.1 in stage 25.0 (TID 1845) (cluster-6311-w-2.c.stable-apogee-366911.internal executor 43): TaskKilled (Stage cancelled)
23/01/10 02:04:47 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 13.1 in stage 25.0 (TID 1855) (cluster-6311-w-0.c.stable-apogee-366911.internal executor 40): TaskKilled (Stage cancelled)
23/01/1

In [52]:
inverted_body.df

{'theoretical': 34848,
 'ahmadu': 894,
 'trawler': 2341,
 'omagbemi': 76,
 'frankokratia': 55,
 'gooyer': 74,
 'nuclear': 57151,
 'bloody': 22010,
 'mckinnon': 2908,
 'distributional': 697,
 'plantas': 271,
 'compsognathus': 101,
 'film2006': 559,
 'sesamoid': 160,
 '5james': 88,
 'baltz': 194,
 'kg60': 163,
 'helvetica': 553,
 'balandin': 154,
 'etem': 87,
 'tatem': 87,
 'mannen': 427,
 'sureness': 120,
 'vällingby': 85,
 'daham': 147,
 'per-erik': 107,
 'sharavathi': 65,
 'bassman': 334,
 'kehot': 63,
 'akros': 73,
 'tarma': 199,
 'nimir': 74,
 'devanāgarī': 113,
 'theretofore': 142,
 "parade's": 383,
 'pin': 16223,
 'biaggi': 590,
 'sefolosha': 76,
 'labora': 106,
 'pridgen': 85,
 '33-17': 78,
 'skratch': 164,
 'maramureş': 117,
 'cxt': 72,
 'dzama': 67,
 '5834': 93,
 "subchannel's": 59,
 'batenburg': 51,
 'slatter': 226,
 'pendulus': 77,
 '9892': 69,
 'warlick': 144,
 'mandakini': 314,
 'an-nasir': 277,
 'streptococcaceae': 53,
 '1963-1972': 77,
 "okamoto's": 94,
 "usaf's": 395,
 '

OM TEST : 

In [7]:
tit_index=pd.read_pickle('/home/dataproc/tit_index.pkl')

UnpicklingError: unpickling stack underflow

## DL OF BODY

In [19]:
def DL(text,id):
  lst_words=[]
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  for i in tokens:
    if i not in all_stopwords:
      lst_words.append(i)
  
  return ((id,len(lst_words)))

In [16]:
doc_text_DL = parquetFile.select("text", "id").rdd

In [ ]:
doc2len = doc_text_DL.map(lambda x:DL(x[0],x[1]))
dict_DL = doc2len.collectAsMap()

In [23]:
import pickle

# Create a dictionary
my_dict = {"a": 1, "b": 2, "c": 3}

# Open a file to write the pickle to
with open("dict_DL.pkl", "wb") as pickle_file:
    # Write the dictionary to the file
    pickle.dump(dict_DL, pickle_file)

## Anchor Text

In [17]:
doc_text_pairs_anchor_text=parquetFile.select("anchor_text", "id").rdd

In [18]:
id2anchor=doc_text_pairs_anchor_text.flatMap(lambda x: x[0])
id2text=id2anchor.distinct().groupByKey().mapValues(lambda x: " ".join(x))

In [19]:
word_counts = id2text.flatMap(lambda x: word_count(x[1], x[0]))

In [20]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

In [ ]:
w2df_d = calculate_df(postings)
w2df_anchor = w2df_d.collectAsMap()

In [ ]:
w2termanchor = calculate_term_total(postings)
dict_term_total_anchor = w2termanchor.collectAsMap()

In [ ]:
_ = partition_postings_and_write(postings).collect()

In [25]:
super_posting_locs_ANCOR = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_ANCOR[k].extend(v)

In [26]:
inverted_index_anchor = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_index_anchor.posting_locs = super_posting_locs_ANCOR
# Add the token - df dictionary to the inverted index
inverted_index_anchor.df = w2df_anchor
#Add the total terms for each term
inverted_index_anchor.term_total = dict_term_total_anchor
# write the global stats out

inverted_index_anchor.write_index('.','anchor_index')
print('FINISH')

FINISH
